In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *
from pbpstats.resources.enhanced_pbp import FieldGoal

data_DIR = "../Shot_Charts/ShotLocationData/"

In [ ]:
def get_loc_data(games_list,player_dict,team_dict):
    game_id, period, clock, seconds_remaining,poss_length = [],[],[],[],[]
    team_id, player1_id= [],[]
    locX, locY, distance, shot_value, shot_type = [],[],[],[],[]
    is_made, is_putback, is_assisted, player2_id  = [],[],[],[]
    pos_store = []
    for game in tqdm(games_list):
        for possession in game.possessions.items:
            for possession_event in possession.events:
                if isinstance(possession_event, FieldGoal):
                    try: 
                        game_id.append(possession_event.game_id)
                        period.append(possession_event.period)
                        clock.append(possession_event.clock)
                        seconds_remaining.append(possession_event.seconds_remaining)
                        poss_length.append(possession_event.seconds_since_previous_event)
                        team_id.append(possession_event.team_id)
                        player1_id.append(possession_event.player1_id)
                        locX.append(possession_event.locX)
                        locY.append(possession_event.locY)
                        distance.append(possession_event.distance)
                        shot_value.append(possession_event.shot_value)
                        shot_type.append(possession_event.shot_type)
                        is_made.append(possession_event.is_made)
                        is_assisted.append(possession_event.is_assisted)
                        if possession_event.is_assisted:
                            player2_id.append(possession_event.player2_id)
                        else:
                            player2_id.append(0)
                        is_putback.append(possession_event.is_putback)
                    except:
                        is_putback.append(False)
    player1_name = np.array([player_dict.get(x,np.nan) for x in player1_id])  
    player2_name = np.array([player_dict.get(x,np.nan) for x in player2_id])  
    team_name = [team['Team'] for team, tID in product(team_dict,team_id) if team['TeamID'] ==tID]
    data = pd.DataFrame({'game_id':game_id,'period':period,'clock':clock,
                        'seconds_remaining':seconds_remaining,'poss_length':poss_length,
                        'team_id':team_id,'team_name':team_name,'player_id':player1_id,
                        'player_name':player1_name,'locX': locX,'locY':locY, 'distance':distance,
                        'shot_value':shot_value,'shot_type':shot_type,'is_made':is_made,
                        'is_assisted':is_assisted,'player_ast_id':player2_id,'player_ast_name':player2_name,
                        'is_putback':is_putback})
    return data

In [ ]:
data_provider = "data_nba"
league = "NBA"
season_type = "Regular Season"
seasons = np.arange(2023,2024,1).astype(str)
for season in seasons:
    games_id = pbp_season(league=league,season_yr=season,season_type=season_type,data_provider=data_provider)
    games_list = pbp_games(games_id,data_provider=data_provider)
    player_dict = get_players_pbp(league=league)
    team_dict = teams.get_teams()
    team_dict = get_teams(league = league)
    data = get_loc_data(games_list,player_dict,team_dict)
    data.to_csv(data_DIR+f"{league}_Shot_Loc_"+season+".csv",index=False)
